## Part 1

In [1]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2024, day=6)

def test(method, input, expected):
    actual = method(input)
    if actual == expected:
        print(f'\t☑ - {method.__name__}({input}) = {expected} = {actual}')
    else:
        print(f'\t☐ - {method.__name__}({input}) = {expected} ≠ {actual}')

In [2]:
puzzle.examples[0].input_data

'....#.....\n.........#\n..........\n..#.......\n.......#..\n..........\n.#..^.....\n........#.\n#.........\n......#...'

In [3]:

def calculate_positions(input:str):
    map = [list(row) for row in input.split()]
    print(map)

    obstacles = []
    direction = None
    guard = None

    # Parse the map
    for y, row in enumerate(map):
        for x, value in enumerate(row):
            if value == '#':
                obstacles.append((x,y))
            elif value == '.':
                continue
            elif value in ['^', '<', '>', 'v']:
                # guard
                guard = (x,y)
                match value:
                    case '^':
                        direction = (0, -1)
                    case '>':
                        direction = (1, 0)
                    case 'v':
                        direction = (0, 1)
                    case '<':
                        direction = (-1, 0)
            else:
                print(f"Didn't know what to make of {value} when parsing the map")

    # Walk the guard around
    positions = set()

    # I'm going to assume that the guard's path is a loop.
    back_to_start = False
    i = 0
    while i < 1000 and not back_to_start:
        i+=1
        positions.add(guard)
        next_ = tuple(sum(x) for x in zip(guard, direction))

        if next_ in obstacles:
            # turn
            # I can't think of a case where the guard will get blocked since you could always go back the way you came?
            # TODO There must be a smart math way to do this, but I'm tired. I'd like to figure it out though.
            match direction:
                case (0, -1): # '^'
                    direction = (1, 0)
                case (1, 0): # '>'
                    direction = (0, 1)
                case (0, 1): # 'v'
                    direction = (-1, 0)
                case (-1, 0): # '<'
                    direction = (0, -1)
        else:
            guard = next_
    
    print(f'{len(positions)}: {positions}')

calculate_positions(puzzle.examples[0].input_data)

[['.', '.', '.', '.', '#', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '.', '.', '#'], ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'], ['.', '.', '#', '.', '.', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '#', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'], ['.', '#', '.', '.', '^', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '.', '.', '#', '.'], ['#', '.', '.', '.', '.', '.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.', '.', '#', '.', '.', '.']]
986: {(7, 673), (7, 17), (7, 682), (7, 920), (7, 26), (7, 264), (7, 691), (7, 929), (7, 35), (7, 273), (7, 938), (7, 282), (7, 520), (7, 947), (7, 291), (7, 529), (7, 300), (7, 538), (7, 547), (7, 785), (7, 129), (7, 556), (7, 794), (4, 2), (7, 138), (7, 803), (7, 147), (7, 385), (3, 6), (7, 812), (7, 156), (7, 394), (7, 821), (7, 165), (7, 403), (7, 412), (7, 650), (7, 421), (7, 659), (7, 668), (7, 906), (7, 12), (7, 250), (7, 677), (7, 915), (7, 21), (7, 259), (7, 686), (7

Ok, something is going wrong. That number is WAY more than 41. Let's get some better debugging.

In [4]:
from dataclasses import dataclass

@dataclass
class Direction:
    symbol:str
    x:int
    y:int

north = Direction('^', 0, -1)
east = Direction('>', 1, 0)
south = Direction('v', 0, 1)
west = Direction('<', -1, 0)

directions = [north, east, south, west]

def draw_map(map_size, guard, direction, visited, obstacles):
    x, y = map_size

    map = []

    for j in range(y):
        row = []
        for i in range(x):
            if (i,j) == guard:
                row.append(direction.symbol)
            elif (i,j) in obstacles:
                row.append('#')
            elif (i,j) in visited:
                row.append('X')
            else:
                row.append('.')
        map.append(row)

    return '\n'.join([''.join(row) for row in map])

def calculate_positions(input:str):
    map = [list(row) for row in input.split()]

    obstacles = []
    direction = None
    guard = None

    # Parse the map
    for y, row in enumerate(map):
        for x, value in enumerate(row):
            if value == '#':
                obstacles.append((x,y))
            elif value == '.':
                continue
            elif value in ['^', '<', '>', 'v']:
                # guard
                guard = (x,y)
                direction = next(filter(lambda d: d.symbol == value, directions))
            else:
                print(f"Didn't know what to make of {value} when parsing the map")

    print(draw_map((len(map[0]), len(map)), guard, direction, [], obstacles))

    # Walk the guard around
    positions = set()

    # I'm going to assume that the guard's path is a loop.
    back_to_start = False
    i = 0
    while i < 80 and not back_to_start:
        i+=1
        positions.add(guard)
        next_ = tuple(sum(x) for x in zip(guard, (direction.x, direction.y)))

        if next_ in obstacles:
            # turn
            # I can't think of a case where the guard will get blocked since you could always go back the way you came?
            # TODO There must be a smart math way to do this, but I'm tired. I'd like to figure it out though.
            indx = directions.index(direction)
            if indx + 1 < len(directions):
                direction = directions[indx+1]
            else:
                direction = directions[0]
        else:
            guard = next_
        
        # print(f'\n{'='*10}')
        # print(draw_map((len(map[0]), len(map)), guard, direction, positions, obstacles))
        # print(f'{'='*10}\n')
        
    print(f'\n{'='*10}')
    print(draw_map((len(map[0]), len(map)), guard, direction, positions, obstacles))
    print(f'{'='*10}\n')
    print(f'{len(positions)}: {positions}')

calculate_positions(puzzle.examples[0].input_data)

....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...

....#.....
....XXXXX#
....X...X.
..#.X...X.
..XXXXX#X.
..X.X.X.X.
.#XXXXXXX.
.XXXXXXX#.
#XXXXXXX..
......#X..

66: {(7, 17), (7, 23), (3, 4), (4, 3), (7, 20), (3, 7), (5, 4), (4, 6), (7, 26), (5, 1), (5, 7), (7, 29), (7, 32), (8, 3), (8, 6), (2, 5), (2, 8), (7, 1), (7, 7), (6, 5), (7, 10), (6, 8), (7, 16), (7, 13), (4, 2), (7, 19), (4, 5), (7, 22), (7, 28), (5, 6), (4, 8), (3, 6), (7, 25), (8, 2), (7, 31), (7, 34), (8, 5), (2, 4), (2, 7), (6, 1), (1, 8), (6, 4), (7, 9), (6, 7), (7, 6), (7, 12), (7, 15), (7, 21), (4, 1), (4, 7), (7, 18), (7, 24), (4, 4), (7, 27), (3, 8), (7, 30), (8, 4), (7, 33), (8, 1), (5, 8), (1, 7), (2, 6), (6, 6), (7, 11), (7, 8), (7, 14)}


OHHHH!!! I'm letting him walk off the map. Woops!

In [5]:
from dataclasses import dataclass

@dataclass
class Direction:
    symbol:str
    x:int
    y:int

north = Direction('^', 0, -1)
east = Direction('>', 1, 0)
south = Direction('v', 0, 1)
west = Direction('<', -1, 0)

directions = [north, east, south, west]

def calculate_positions(input:str):
    map = [list(row) for row in input.split()]

    obstacles = []
    direction = None
    guard = None

    # Parse the map
    for y, row in enumerate(map):
        for x, value in enumerate(row):
            if value == '#':
                obstacles.append((x,y))
            elif value == '.':
                continue
            elif value in ['^', '<', '>', 'v']:
                # guard
                guard = (x,y)
                direction = next(filter(lambda d: d.symbol == value, directions))
            else:
                print(f"Didn't know what to make of {value} when parsing the map")

    map_x_size = len(map[0])
    map_y_size = len(map)

    # Walk the guard around
    positions = set()

    # I'm going to assume that the guard's path is a loop.
    outside_of_map = False
    i = 0
    while i < 10000 and not outside_of_map:
        i+=1
        positions.add(guard)
        next_ = tuple(sum(x) for x in zip(guard, (direction.x, direction.y)))

        if next_ in obstacles:
            # turn
            # I can't think of a case where the guard will get blocked since you could always go back the way you came?
            # TODO There must be a smart math way to do this, but I'm tired. I'd like to figure it out though.
            indx = directions.index(direction)
            if indx + 1 < len(directions):
                direction = directions[indx+1]
            else:
                direction = directions[0]
        elif next_[0] >= map_x_size or next_[1] >= map_y_size:
            outside_of_map = True
        else:
            guard = next_
        
        # print(f'\n{'='*10}')
        # print(draw_map((len(map[0]), len(map)), guard, direction, positions, obstacles))
        # print(f'{'='*10}\n')

    #print(f'{len(positions)}: {positions}')
    return len(positions)

In [6]:
calculate_positions(puzzle.examples[0].input_data)

41

In [7]:
calculate_positions(puzzle.input_data)

4964

## Part 2

Ohhh, discovered a bug! They can walk off the screen to the top and I missed that case (and the left)

In [8]:
from dataclasses import dataclass

@dataclass
class Direction:
    symbol:str
    x:int
    y:int

north = Direction('^', 0, -1)
east = Direction('>', 1, 0)
south = Direction('v', 0, 1)
west = Direction('<', -1, 0)

directions = [north, east, south, west]

@dataclass
class Guard:
    direction:Direction
    position:tuple[int,int]

def check_walk_outside(map_size, initial_guard, initial_direction, obstacles):
    map_x_size, map_y_size = map_size
    guard = Guard(initial_direction, initial_guard)

    # Walk the guard around
    positions = set()

    outside_of_map = False
    # Smarter than the last 10k would be to keep the direction of the guard and the position in the positions. Lets see if I have to do that though
    i = 0
    while i < 10000 and not outside_of_map:
        i+=1
        positions.add(guard.position)
        next_ = tuple(sum(x) for x in zip(guard.position, (guard.direction.x, guard.direction.y)))

        if next_ in obstacles:
            # turn
            # I can't think of a case where the guard will get blocked since you could always go back the way you came?
            # TODO There must be a smart math way to do this, but I'm tired. I'd like to figure it out though.
            indx = directions.index(guard.direction)
            if indx + 1 < len(directions):
                guard.direction = directions[indx+1]
            else:
                guard.direction = directions[0]
        elif next_[0] >= map_x_size or next_[1] >= map_y_size or next_[0] < 0 or next_[1] < 0:
            outside_of_map = True
        else:
            guard.position = next_

    # This return is to support debugging. If this was real, I would do this totally differently!
    return (outside_of_map, positions, guard)

def calculate_positions(input:str):
    map = [list(row) for row in input.split()]

    obstacles = []
    direction = None
    guard = None

    # Parse the map
    for y, row in enumerate(map):
        for x, value in enumerate(row):
            if value == '#':
                obstacles.append((x,y))
            elif value == '.':
                continue
            elif value in ['^', '<', '>', 'v']:
                # guard
                guard = (x,y)
                direction = next(filter(lambda d: d.symbol == value, directions))
            else:
                print(f"Didn't know what to make of {value} when parsing the map")

    map_x_size = len(map[0])
    map_y_size = len(map)

    successful_obstacles = 0

    for y in range(len(map)):
        for x in range(len(row)):
            
            if (x, y) in obstacles:
                continue
            added_obstacles = obstacles.copy()
            added_obstacles.append((x,y))
            outside_of_map, positions, g = check_walk_outside((map_x_size, map_y_size), guard, direction, added_obstacles)
            if not outside_of_map:
                # print(f'\n{'='*10}')
                # print(draw_map((map_x_size, map_y_size), guard, direction, [], obstacles))
                # print(f'{'='*10}')
                # print(draw_map((map_x_size, map_y_size), guard, direction, [], added_obstacles))
                # print(f'\n{'='*10}')
                # print(draw_map((map_x_size, map_y_size), g.position, g.direction, positions, added_obstacles))
                # print(f'{'='*10}\n')
                successful_obstacles += 1

        
       

    #print(f'{len(positions)}: {positions}')
    return successful_obstacles

In [9]:
calculate_positions(puzzle.examples[0].input_data)

6

In [10]:
calculate_positions(puzzle.input_data)

KeyboardInterrupt: 

Will I ever learn that Advent of Code is designed to keep me from brute forcing? :D

In [11]:
@dataclass
class Guard:
    direction:Direction
    position:tuple[int,int]

    def __hash__(self):
        return hash((self.direction.symbol, self.position))

    def __eq__(self, other):
        return self.direction.symbol == other.direction.symbol and self.position == other.position

def calculate_positions(input:str):
    map = [list(row) for row in input.split()]

    obstacles = []
    direction = None
    guard = None

    # Parse the map
    for y, row in enumerate(map):
        for x, value in enumerate(row):
            if value == '#':
                obstacles.append((x,y))
            elif value == '.':
                continue
            elif value in ['^', '<', '>', 'v']:
                # guard
                guard = (x,y)
                direction = next(filter(lambda d: d.symbol == value, directions))
            else:
                print(f"Didn't know what to make of {value} when parsing the map")

    map_x_size = len(map[0])
    map_y_size = len(map)

    successful_obstacles = 0

    for y in range(len(map)):
        for x in range(len(row)):
            
            if (x, y) in obstacles:
                continue
            added_obstacles = obstacles.copy()
            added_obstacles.append((x,y))
            outside_of_map, positions, g = check_walk_outside((map_x_size, map_y_size), guard, direction, added_obstacles)
            if not outside_of_map:
                # print(f'\n{'='*10}')
                # print(draw_map((map_x_size, map_y_size), guard, direction, [], obstacles))
                # print(f'{'='*10}')
                # print(draw_map((map_x_size, map_y_size), guard, direction, [], added_obstacles))
                # print(f'\n{'='*10}')
                # print(draw_map((map_x_size, map_y_size), g.position, g.direction, positions, added_obstacles))
                # print(f'{'='*10}\n')
                successful_obstacles += 1

        
       

    #print(f'{len(positions)}: {positions}')
    return successful_obstacles

def check_walk_outside(map_size, initial_guard, initial_direction, obstacles):
    map_x_size, map_y_size = map_size
    guard = Guard(initial_direction, initial_guard)

    # Walk the guard around
    positions = set()

    outside_of_map = False
    looped = False
    # Smarter than the last 10k would be to keep the direction of the guard and the position in the positions. Lets see if I have to do that though
    i = 0
    while i < 10000 and not outside_of_map and not looped:
        i+=1
        positions.add(guard)
        next_ = Guard(guard.direction, tuple(sum(x) for x in zip(guard.position, (guard.direction.x, guard.direction.y))))

        if next_ in positions:
            looped = True
        elif next_.position in obstacles:
            # turn
            # I can't think of a case where the guard will get blocked since you could always go back the way you came?
            # TODO There must be a smart math way to do this, but I'm tired. I'd like to figure it out though.
            indx = directions.index(guard.direction)
            if indx + 1 < len(directions):
                guard.direction = directions[indx+1]
            else:
                guard.direction = directions[0]
        elif next_.position[0] >= map_x_size or next_.position[1] >= map_y_size or next_.position[0] < 0 or next_.position[1] < 0:
            outside_of_map = True
        else:
            guard = next_

    # This return is to support debugging. If this was real, I would do this totally differently!
    return (outside_of_map, positions, guard)

calculate_positions(puzzle.examples[0].input_data)


6

In [13]:
calculate_positions(puzzle.input_data)

1740

If this was more than just a game, I'd get a profiler out and see where I'm slow. I can think of two things to speed it up. I could take the walk from part one and only check obstacles in those paths. I wonder if it's possible to use the obstacle I've placed and guess how to start the walk from there. Might make it faster to understand if there is a loop or not.